In [1]:
from bs4 import BeautifulSoup
import requests
import time

def scraping(topic):
    url = 'https://news.yahoo.co.jp/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    list_item = soup.find_all('li')
    #もしかしたらエラーの原因になるかも
    list_item = list_item[12:21]
    
    # <a>要素からトピックを取得
    a_contents = [li.a for li in list_item]
    
    articles_dict = []
    topic = topic 
    #トピックのURL取得
    for i, a_content in enumerate(a_contents):
        if topic in a_content.text:
            categories_num = i
    
    a_content = a_contents[categories_num]
    
    topic_url = a_content['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    #クラス名が変わる！！
    elements = soup.select('.sc-dvXYtj.iduHXF')
    
    elements = elements[:5]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        url = element.a['href']
        title = element.text[1:]
        news_urls.append(url)
        news_titles.append(title)
        time.sleep(0.2)
    
    news_text = []
    #ニュースの本文を取得
    for news_url in news_urls:
        r = requests.get(news_url)
        soup = BeautifulSoup(r.text, "html.parser")
        #クラス名が変わるので注意！！ 
        elements = soup.select('.sc-gDyJDg.dmrAcv.yjSlinkDirectlink.highLightSearchTarget')
    
        text = ""
    
        #ニュース記事の本文が配列で区切られていた場合
        for i in range(0, 8):
            try:
                if (i == 0):
                    text = elements[0].text
                else:
                    text += elements[i].text
            except:
                pass
    
        #ページが1つだけのとき、エラーが出るため
        try:
            #クラス名が変わるので注意！！ （取得例：２ページ)
            page_element = soup.select('.sc-ihRHuF.kFnlxQ')
            page = page_element[0].text         
            page_str = ''.join(filter(str.isdigit, page))
            page_num = int(page_str)
        except:
            page_num = 1
            
        #ページ数が複数のとき、ページ遷移し本文を取得
        if (page_num > 1):
            for i in range(2, page_num + 1):
                news_url_page = news_url + "?page=" + str(page_num)
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                #クラス名が変わるので注意！！ 
                elements = soup.select('.sc-gDyJDg.dmrAcv.yjSlinkDirectlink.highLightSearchTarget')
                #ニュース記事の本文が配列で区切られていた場合
                for i in range(0, 8):
                    try:
                        text += elements[i].text
                    except:
                        pass
                        
        news_text.append(text)
    
        time.sleep(1)

    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles

In [4]:
def clean_data(articles):
    for i in range(len(articles)):
        articles[i]['title'] = articles[i]['title'].replace('\u3000', '') 
        articles[i]['title'] = articles[i]['title'].lstrip('0123456789')
        articles[i]['text'] = articles[i]['text'].replace('\u3000', '')
        articles[i]['text'] = articles[i]['text'].replace('\n', '')

    return articles

In [2]:
a = scraping('国際')

In [3]:
a

[{'title': 'ウクライナ軍の南部戦線司令官、反転攻勢で一定の成果と報告CNN.co.jp7/29(土)10:14',
  'URL': 'https://news.yahoo.co.jp/articles/640a31b98c18554f0837af450d17a8f0702e3f85',
  'text': '（ＣＮＮ）\u3000ウクライナの反転攻勢がより積極的な新局面に入りつつあるとみられる中、南部戦線を指揮するウクライナ軍のオレクサンドル・タルナフスキー司令官は２９日までに、自身の部隊がロシア軍の撃退に一定の成果を上げていると報告した。\n\n【映像】ロシア兵が退却する様子か、映像捉える\n\nタルナフスキー氏は「防衛軍は組織的に敵を押し返しており、一定の成功を収めている」と述べた。\n\nさらにウクライナ軍が２７日に制圧を主張した南東部の村を挙げ、「特にドネツク州のスタロマイオルスケを解放し、陣地を固めているところだ」とした。\n\nタルナフスキー氏が指揮する地域にはドネツク、ザポリージャ両州の南部が含まれる。こうした地域では激しい戦闘が進行中だという。\n\nタルナフスキー氏によると、ロシアは過去１日の間にウクライナ軍の陣地に少なくとも１７回の攻撃を仕掛け、火砲を数百回発射した。一方、ウクライナの砲兵部隊は攻勢に出ており、同じ期間に１５００回超の射撃任務を実施したという。\n\n戦況に関するウクライナ、ロシア両国の主張を検証するのは不可能だが、ウクライナはペースが上がらなかった数カ月を経て、反転攻勢を強化しているとみられる。ウクライナ当局は今週、ＣＮＮの取材に対し、南部戦線に新たな兵員を投入していることを明らかにした。\n\n２７日には、タルナフスキー氏の報告にあるスタロマイオルスケ村にウクライナ兵が立っている様子を収めた動画が公開された。\n\nＳＮＳ上では２８日、「竜の歯」と呼ばれるロシアの長大な防衛線にウクライナ軍が初めて到達したとみられる動画も拡散した。竜の歯はコンクリート製の対戦車障害物で、ロシアがウクライナ南部の領土奪還作戦を遅らせるために敷いた多層防衛のひとつ。'},
 {'title': '12万年で1番\u3000命を脅かす猛暑\u3000気温上昇がもたらす暑さより怖いもの…【報道1930】TBS NEWS DIG Power